In [30]:
import shap
import pickle
import pandas
import sklearn
import pkgutil
import os
import importlib, inspect
import tensorflow
import lightgbm
import xgboost

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

# Data Preparation

In [3]:
class featureEngineeringStage():
    def __init__(self, columns, selection):
        self.columns = columns
        self.selection = selection
    
    def transform(self, X, y=None):
        """Transform columns of X using LabelEncoder.
        """
        output = X.copy()
        for col in self.columns:
            output[col] = LabelEncoder().fit_transform(output[col])
        return output[selection]
    
    def fit(self, X, y=None):
        return self

In [4]:
save_dir = "./models"

In [45]:
insurance_training = pickle.load(open("./data/pickle_pandas_tabular_insurance_training.sav", "rb"))
insurance_testing = pickle.load(open("./data/pickle_pandas_tabular_insurance_testing.sav", "rb"))
insurance_testing.describe()

X_train = insurance_training.drop("charges", axis=1)
y_train = insurance_training[["charges"]]
X_test = insurance_testing.drop("charges", axis=1)
y_test = insurance_testing[["charges"]]

regression_background = X_test.sample(50)
regression_test = X_test.sample(10)

##

compas_training = pickle.load(open("./data/pickle_pandas_tabular_compas_training.sav", "rb"))
compas_testing = pickle.load(open("./data/pickle_pandas_tabular_compas_testing.sav", "rb"))

X_train = compas_training.drop("two_year_recid", axis=1)
y_train = compas_training[["two_year_recid"]]
X_test = compas_testing.drop("two_year_recid", axis=1)
y_test = compas_testing[["two_year_recid"]]

bc_background = X_test.sample(50)
bc_test = X_test.sample(10)

##

loan_training = pickle.load(open("./data/pickle_pandas_tabular_loan_training.sav", "rb")).sample(5000)
loan_testing = pickle.load(open("./data/pickle_pandas_tabular_loan_testing.sav", "rb")).sample(2000)

X_train = loan_training.drop("Interest_Rate", axis=1)
y_train = loan_training[["Interest_Rate"]]
X_test = loan_testing.drop("Interest_Rate", axis=1)
y_test = loan_testing[["Interest_Rate"]]

mc_background = X_test.sample(50)
mc_test = X_test.sample(10)

##

regression_pipeline = pickle.load(open("./data/pickle_pandas_tabular_insurance_pipeline_testing.sav".format(save_dir), "rb"))
rp_background = regression_pipeline.sample(50)
rp_test = regression_pipeline.sample(10)

##
bc_pipeline = pickle.load(open("./data/pickle_pandas_tabular_compas_pipeline_testing.sav".format(save_dir), "rb"))
bc_pipeline_background = bc_pipeline.sample(50)
bc_pipeline_test = bc_pipeline.sample(10)

## BC, Regression, MC


In [46]:
for root, dirs, files in os.walk(save_dir):
    for filename in files:
        name = os.path.join(root, filename)
        
        if "tensorflow" in name and name.endswith(".sav"):
            print("Processing ", name)
            model = tensorflow.keras.models.load_model(name)
        
        elif not name.endswith(".py") and not name.endswith("zip") and  "tensorflow" not in name:
            print("Processing ", name)
            model = pickle.load(open(name, "rb"))
        
        else:
            continue
            
        def predict_helper(data):
            return model.predict(data)
        
        print("processing for: ", model)
        
        if "pipeline" in name:
            continue
        elif "regression" in name:
            background = regression_background
            test = regression_test
        elif "binary_classification" in name:
            background = bc_background
            test = bc_test
        elif "multiclass_classification" in name:
            background = mc_background
            test = mc_test
            
            
        # cater for xgboost
        if "xgboost" in name:
            background = xgboost.DMatrix(background, background)
            test = xgboost.DMatrix(test, test)
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(test)
            
        else:
            explainer = shap.KernelExplainer(predict_helper, background)
            shap_values = explainer.shap_values(test)

Processing  ./models/xgboost/1.7.5/regression_insurance_squarederror_xgboost.core.Booster.sav
processing for:  <xgboost.core.Booster object at 0x7f4cec456920>
Processing  ./models/xgboost/1.7.5/binary_classification_compas_xgboost.sklearn.XGBRFClassifier.sav
processing for:  XGBRFClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bytree=None,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
                grow_policy=None, importance_type=None,
                interaction_constraints=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=None, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=None, num_parallel_tree=None,
                objective='binary:logistic', pred

100%|███████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.41it/s]


Processing  ./models/sklearn/1.2.2/multiclass_classification_loan_sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.sav
processing for:  QuadraticDiscriminantAnalysis()


100%|███████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.20it/s]


Processing  ./models/sklearn/1.2.2/multiclass_classification_loan_sklearn.discriminant_analysis.LinearDiscriminantAnalysis.sav
processing for:  LinearDiscriminantAnalysis()


...

100%|███████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.26it/s]


Processing  ./models/lightgbm/3.3.5/regression_insurance_tweedie_lightgbm.basic.Booster.sav
processing for:  <lightgbm.basic.Booster object at 0x7f4cec40f910>


100%|███████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.55it/s]


Processing  ./models/lightgbm/3.3.5/regression_insurance_huber_lightgbm.basic.Booster.sav
processing for:  <lightgbm.basic.Booster object at 0x7f4ce778e4a0>


100%|███████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.85it/s]


## Pipeline Test